## **Entregable 1 - Fernando Martínez (comisión Jueves)**

In [ ]:
# Instalo las bibliotecas necesarias
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Importo las bibliotecas necesarias:
import requests
import json
import pandas as pd
import psycopg2
from io import StringIO

In [ ]:
# URL de la API para obtener los datos diarios de COVID-19 para Estados Unidos
url = 'https://data.cdc.gov/resource/9mfq-cb36.json'

# Realizo la solicitud HTTP a la API
response = requests.get(url)

# Obtengo los datos en formato JSON
data = response.json()

# Selecciono las columnas relevantes (al menos 10 variables)
columnas = ['submission_date', 'tot_cases', 'tot_death', 'new_case', 'new_death','consent_cases', 'consent_deaths', 'conf_death', 'conf_cases', 'pnew_case']
datos = []

# Extraigo los datos para cada día
for registro in data:
    registro_seleccionado = {columna: registro[columna] for columna in columnas if columna in registro}
    datos.append(registro_seleccionado)

# Creo el dataframe de Pandas
df = pd.DataFrame(datos)

# Muestro el dataframe
print(df)

             submission_date tot_cases tot_death new_case new_death  \
0    2021-03-11T00:00:00.000    297229      4851      0.0       0.0   
1    2021-12-01T00:00:00.000    163565      1907    589.0       9.0   
2    2022-01-02T00:00:00.000        11         0      0.0       0.0   
3    2021-11-22T00:00:00.000    841461     16377    703.0       7.0   
4    2022-05-30T00:00:00.000    251425      1252      0.0       0.0   
..                       ...       ...       ...      ...       ...   
995  2021-12-16T00:00:00.000    875502      9841   2429.0      62.0   
996  2021-11-26T00:00:00.000    664667     11223    751.0       0.0   
997  2021-05-03T00:00:00.000    723443     13323    797.0       1.0   
998  2020-12-10T00:00:00.000    225946      1016   3401.0      21.0   
999  2020-06-02T00:00:00.000     18145      1046    322.0      13.0   

    consent_cases consent_deaths conf_cases pnew_case conf_death  
0           Agree            N/A   241035.0         0        NaN  
1           A

In [ ]:
import psycopg2
from io import StringIO

# Conexión a Amazon Redshift
host = 'data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
port = 5439
database = 'data-engineer-database'
user = 'fgmartinez87_coderhouse'
password = '7c92hMs3M1'

# Creo la conexión a Amazon Redshift
conn = psycopg2.connect(
    host=host,
    port=port,
    database=database,
    user=user,
    password=password
)



In [ ]:
# Creo la tabla en Redshift
create_table_query = '''
CREATE TABLE IF NOT EXISTS covid_data (
    submission_date DATE,
    tot_cases INT,
    tot_death INT,
    new_case FLOAT,
    new_death FLOAT,
    consent_cases VARCHAR(10),
    consent_deaths VARCHAR(10),
    conf_death FLOAT,
    conf_cases FLOAT,
    pnew_case FLOAT
);
'''
with conn.cursor() as cur:
    cur.execute(create_table_query)
    conn.commit()


## **Entregable 2 - Fernando Martínez (comisión Jueves)**

In [ ]:
# Inserto los datos del df
insert_query = 'INSERT INTO covid_data VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'
data_to_insert = [tuple(row) for row in df.values]
with conn.cursor() as cur:
    cur.executemany(insert_query, data_to_insert)
    conn.commit()

In [ ]:
# Realizo la consulta en Redshift para ver si se cargó la data
query = 'SELECT * FROM covid_data;'
df_result = pd.read_sql_query(query, conn)

# Muestro el resultado
print(df_result)

    submission_date  tot_cases  tot_death  new_case  new_death consent_cases  \
0        2021-03-11     297229       4851       0.0        0.0         Agree   
1        2021-12-01     163565       1907     589.0        9.0         Agree   
2        2022-01-02         11          0       0.0        0.0           NaN   
3        2021-11-22     841461      16377     703.0        7.0         Agree   
4        2022-05-30     251425       1252       0.0        0.0           N/A   
..              ...        ...        ...       ...        ...           ...   
995      2021-12-16     875502       9841    2429.0       62.0         Agree   
996      2021-11-26     664667      11223     751.0        0.0         Agree   
997      2021-05-03     723443      13323     797.0        1.0     Not agree   
998      2020-12-10     225946       1016    3401.0       21.0         Agree   
999      2020-06-02      18145       1046     322.0       13.0         Agree   

    consent_deaths  conf_death  conf_ca

<ipython-input-30-c7ea476a5afb>:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql_query(query, conn)


In [ ]:
# Cuento la cantidad de registros para validar
with conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM covid_data")
    count = cur.fetchone()[0]
    print(f"Cantidad de registros en la tabla covid_data: {count}")

Cantidad de registros en la tabla covid_data: 1000


In [ ]:
# Quiero validar que no hayan duplicados
with conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM (SELECT DISTINCT * FROM covid_data) AS unique_records;")
    count = cur.fetchone()[0]
    print(f"Cantidad de registros únicos en la tabla covid_data: {count}")

Cantidad de registros únicos en la tabla covid_data: 999


Existe un resgistro duplicado

In [ ]:
# Query para borrar la tabla
# with conn.cursor() as cur:
#    truncate_query = 'DROP TABLE covid_data;'
#    cur.execute(truncate_query)
# conn.commit()

In [ ]:
# Inserto los mismos registros dos veces para generar más duplicados
insert_query = 'INSERT INTO covid_data VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'
data_to_insert = [tuple(row) for row in df.values]
with conn.cursor() as cur:
    cur.executemany(insert_query, data_to_insert)
    conn.commit()

In [ ]:
# Cuento la cantidad de registros para validar
with conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM covid_data")
    count = cur.fetchone()[0]
    print(f"Cantidad de registros en la tabla covid_data: {count}")

Cantidad de registros en la tabla covid_data: 2000


In [ ]:
# Query para eliminar duplicados
with conn.cursor() as cur:
    # Creo una nueva tabla temporal sin duplicados
    cur.execute("CREATE TABLE covid_data_temp AS SELECT DISTINCT * FROM covid_data;")

    # Elimino la tabla original
    cur.execute("DROP TABLE covid_data;")

    # Renombro la tabla temporal como la tabla original
    cur.execute("ALTER TABLE covid_data_temp RENAME TO covid_data;")

    # Confirmo los cambios
    conn.commit()

    # Obtengo la cantidad de registros después de eliminar duplicados
    cur.execute("SELECT COUNT(*) FROM covid_data;")
    count = cur.fetchone()[0]
    print(f"Cantidad de registros en la tabla covid_data después de eliminar duplicados: {count}")

Cantidad de registros en la tabla covid_data después de eliminar duplicados: 999
